Absolute magnitude is defined to be the apparent magnitude an object would have if it were located at a distance of 10 parsecs.
In astronomy, absolute magnitude (M) is a measure of the luminosity of a celestial object on an inverse logarithmic astronomical magnitude scale.

In [1]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz
from astropy.cosmology import Planck18 as cosmo
from scipy import signal
import os 
import csv
import resource


from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.particle.stars import sample_sfhz
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.particles import CoordinateGenerator
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds

from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.kernel_functions import Kernel

from synthesizer.conversions import lnu_to_absolute_mag

current_directory = os.getcwd()
print("Current directory:", current_directory)
# Start the timer
start_time = time.time()


/tmp/ipykernel_4458/854112820.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Current directory: /home/jovyan/camels/play/synth-play/LH


In [2]:

# Alternative method for LF:
# try this method again, but using AB mag instead of mass, and suply your own bins (up to -17, say)
def calc_lf(ab_mag, volume, massBinLimits):
# OG:        hist, dummy = np.histogram(np.log10(mstar), bins = massBinLimits)
        hist, dummy = np.histogram(ab_mag, bins = massBinLimits)
        hist = np.float64(hist)
        phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])
        phi_sigma = (np.sqrt(hist) / volume) /\
                    (massBinLimits[1] - massBinLimits[0]) # Poisson errors
        return phi, phi_sigma, hist
    


phi_arr =[] #phi
phi_sigma_arr =[] # phi_sigma
hist_arr = [] # hist
z_arr = [] #redshift_074, 
abs_mag_arr = [] #absolute mag (th filter)
Vcom_arr = [] # comoving vol
massBinLimits_arr = []


len(phi_arr)

In [3]:
#directories = [f'LH_{i}' for i in range(1000)]
# TEST:
directories = [f'LH_{i}' for i in range(503,628)]
directories

['LH_503',
 'LH_504',
 'LH_505',
 'LH_506',
 'LH_507',
 'LH_508',
 'LH_509',
 'LH_510',
 'LH_511',
 'LH_512',
 'LH_513',
 'LH_514',
 'LH_515',
 'LH_516',
 'LH_517',
 'LH_518',
 'LH_519',
 'LH_520',
 'LH_521',
 'LH_522',
 'LH_523',
 'LH_524',
 'LH_525',
 'LH_526',
 'LH_527',
 'LH_528',
 'LH_529',
 'LH_530',
 'LH_531',
 'LH_532',
 'LH_533',
 'LH_534',
 'LH_535',
 'LH_536',
 'LH_537',
 'LH_538',
 'LH_539',
 'LH_540',
 'LH_541',
 'LH_542',
 'LH_543',
 'LH_544',
 'LH_545',
 'LH_546',
 'LH_547',
 'LH_548',
 'LH_549',
 'LH_550',
 'LH_551',
 'LH_552',
 'LH_553',
 'LH_554',
 'LH_555',
 'LH_556',
 'LH_557',
 'LH_558',
 'LH_559',
 'LH_560',
 'LH_561',
 'LH_562',
 'LH_563',
 'LH_564',
 'LH_565',
 'LH_566',
 'LH_567',
 'LH_568',
 'LH_569',
 'LH_570',
 'LH_571',
 'LH_572',
 'LH_573',
 'LH_574',
 'LH_575',
 'LH_576',
 'LH_577',
 'LH_578',
 'LH_579',
 'LH_580',
 'LH_581',
 'LH_582',
 'LH_583',
 'LH_584',
 'LH_585',
 'LH_586',
 'LH_587',
 'LH_588',
 'LH_589',
 'LH_590',
 'LH_591',
 'LH_592',
 'LH_593',

In [4]:
len(directories)

125

In [ ]:
# Try data frame instead
grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
grid_dir = "/home/jovyan/"
grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)
    
# Create an empty dictionary to store your data
data_dict = {}

for LH_X in directories:

    # get grid for all galaxies
    # gives grid of metalicity and age which is used to map on our camels galaxies

    # get gals
    dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/LH/' + LH_X
    gals_074 = load_CAMELS_IllustrisTNG(
        dir_,
        snap_name='snapshot_074.hdf5', 
        fof_name='groups_074.hdf5',
    )

    cat_074 = dir_+'/groups_074.hdf5'
    # open file
    f = h5py.File(cat_074, 'r')

    # read different attributes of the header
    boxSize_074 = f['Header'].attrs[u'BoxSize']/1e3 #Mpc/h
    redshift_074 = f['Header'].attrs[u'Redshift']


    # Filter galaxies to only include those with 100 or more star particles
    ## NEED TO ADD FILTER TO GET RID OF ANY GALAXIES WITH LESS THAN 100 star Particles!! This is the resolution limit!!
    # Chris: when you load your galaxies in could you do a filter for those with at least 100 star particles? that's the resolution limit
    gals_074 = [gal for gal in gals_074 if len(gal.stars.ages) >= 100]
    spec_list = []
    # Lets work with z=0 so gals_025
    for i in gals_074:
        gal = i
        # get_spectra_incident An Sed object containing the stellar spectra
        spec = gal.stars.get_spectra_incident(grid)
        spec.get_fnu0()
        spec_list.append(spec)

    # combine
    seds = combine_list_of_seds(spec_list)
    filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
    filt_lst = [filt1]
    seds.lnu # rest frame lumd
    seds.get_photo_luminosities(filt_lst)
    seds.photo_luminosities.photo_luminosities
    abs_mag = lnu_to_absolute_mag(seds.photo_luminosities.photo_luminosities)
    abs_mag_th = abs_mag[0]
    # next steps, get luminosity function for these magnitudes
    # co-moving volume: BoxSize_025 and redshift:
    little_h =  0.6711
    Vphys = (boxSize_074/little_h )**3
    Vcom = Vphys * ((1+redshift_074)**3)

    massBinLimits = np.arange(-22, -16, 0.5)
    phi, phi_sigma, hist = calc_lf(abs_mag_th, Vcom, massBinLimits)
    # NOTE: 074 is the same redshift as CV_0/025
    massBinLimits = massBinLimits[:-1]

    # Store the calculated values in the dictionary with LH_X as the key
    data_dict[LH_X] = {
        'phi': phi,
        'phi_sigma': phi_sigma,
        'hist': hist,
        'massBinLimits': massBinLimits
    }

    # Get memory usage in bytes
    memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    print("Memory usage during loop: ", memory_usage/1000000, "GB")

# done

/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1938: RuntimeWarning: invalid value encountered in divide
  out_arr = func(


Memory usage during loop:  1.263604 GB


/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1813: RuntimeWarning: divide by zero encountered in log10
  out_arr = func(np.asarray(inp), out=out_func, **kwargs)


Memory usage during loop:  1.563132 GB
Memory usage during loop:  1.775968 GB
Memory usage during loop:  1.775968 GB
Memory usage during loop:  1.81222 GB
Memory usage during loop:  2.03134 GB


/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1813: RuntimeWarning: invalid value encountered in log10
  out_arr = func(np.asarray(inp), out=out_func, **kwargs)


Memory usage during loop:  2.08068 GB
Memory usage during loop:  2.08068 GB
Memory usage during loop:  2.08068 GB
Memory usage during loop:  2.08068 GB


In [ ]:
import pickle
# Save the dictionary containing all data
with open('/home/jovyan/output_test/LH_503-628.pkl', 'wb') as f:
    pickle.dump(data_dict, f)

# will do latest LH_ set in loop as a test
label_z = 'z = 0.46'


# Plot the luminosity function
plt.errorbar(massBinLimits, phi, yerr=phi_sigma, fmt='o', color='blue',label=label_z)
plt.xlabel('Absolute Magnitude (AB)')
plt.ylabel('Number Density (Mpc^-3 mag^-1)')
plt.yscale('log')

plt.title('Luminosity Function XMM-OM filter')
plt.grid(True)
plt.show()

In [ ]:
# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
# Get memory usage in bytes
memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
print("Memory usage end:", memory_usage/1000000, "GB")
